In [4]:
import gmaps
from config_gmaps import g_key
import pandas as pd 
import requests

In [5]:
challenge_gmaps_df = pd.read_csv('data/WeatherPy_challenge.csv')
challenge_gmaps_df.dtypes

City_ID                       int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [6]:
# user prompts for input min and max temp range and whether rain/snow or not
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    
raining_prompt = input("Do you want it to be raining? (yes/no) ")
snowing_prompt = input("Do you want it to be snowing? (yes/no) ")

# Filter DataFrames based on input and nested decision statements, and logical expressions
if (raining_prompt == 'yes') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']==0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'yes') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']==0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
else:
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!=0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!=0) &\
                                                             (challenge_gmaps_df['Max Temp']<=max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>=min_prompt_temp)]
    
preferred_challenges_df.head()

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,kalmunai,LK,2019-12-31 06:04:05,7.42,81.82,81.27,77,86,9.22,overcast clouds,0.0,0.0
2,2,bambous virieux,MU,2019-12-31 06:04:06,-20.34,57.76,81.00,88,75,12.75,shower rain,0.0,0.0
3,3,atuona,PF,2019-12-31 06:04:06,-9.80,-139.03,80.35,75,1,14.94,clear sky,0.0,0.0
5,5,vaini,IN,2019-12-31 06:04:07,15.34,74.49,75.20,44,40,2.66,scattered clouds,0.0,0.0
12,12,busselton,AU,2019-12-31 06:04:09,-33.64,115.35,84.00,20,0,16.98,clear sky,0.0,0.0


In [11]:
preferred_challenges_df.count()

City_ID                     124
City                        124
Country                     124
Date                        124
Lat                         124
Lng                         124
Max Temp                    124
Humidity                    124
Cloudiness                  124
Wind Speed                  124
Current Description         124
Rain inches (last 3 hrs)    124
Snow inches (last 3 hrs)    124
dtype: int64

In [8]:
# clean up the DataFrame by dropna() funtion
clean_preferred_challenges_df = preferred_challenges_df.dropna()
clean_preferred_challenges_df.count()
# make a copy for a new DataFrame 
challenge_hotel_df = clean_preferred_challenges_df[["City", "Country","Max Temp",  "Current Description","Lat", "Lng"]].copy()
challenge_hotel_df['Hotel Name'] = ''

# search nearest hotel name by using google place API 
for index, row in challenge_hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params = {'key': g_key, 'type':'lodging', 'radius' : 5000}
    params['location'] = f"{lat},{lng}"
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        challenge_hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

# Save and upload the new DataFrame as csv file
challenge_hotel_df.to_csv(path_or_buf='data/weatherPy_vacation.csv', index_label='City_ID')

challenge_hotel_df.head(10)

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,kalmunai,LK,81.27,overcast clouds,7.42,81.82,VS Villa
2,bambous virieux,MU,81.00,shower rain,-20.34,57.76,Casa Tia Villa
3,atuona,PF,80.35,clear sky,-9.80,-139.03,Villa Enata
5,vaini,IN,75.20,scattered clouds,15.34,74.49,Dandeli Lake County
12,busselton,AU,84.00,clear sky,-33.64,115.35,Observatory Guest House
15,arraial do cabo,BR,75.07,clear sky,-22.97,-42.02,Pousada Porto Praia
18,barcelos,BR,73.40,overcast clouds,-0.97,-62.93,Barco Hotel Princesa Amazonia
20,puerto ayora,EC,73.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
26,mahibadhoo,MV,81.91,scattered clouds,3.78,72.97,Omadhoo Inn
27,avarua,CK,77.00,overcast clouds,-21.21,-159.78,Paradise Inn


In [9]:
# create a  marker layer map with a pop-up info box by gmaps dependence
gmaps.configure(api_key = g_key)
locations = challenge_hotel_df[['Lat','Lng']]
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>'''

hotel_info = [info_box_template.format(**row) for index, row in challenge_hotel_df.iterrows() ]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))